In [1]:
!pip install -e ../../

Obtaining file:///C:/Users/netxph/Projects/mlpouch
  Attempting uninstall: mlpouch
    Found existing installation: mlpouch 0.0.1
    Uninstalling mlpouch-0.0.1:
      Successfully uninstalled mlpouch-0.0.1
  Running setup.py develop for mlpouch


In [2]:
import pandas as pd
from mlpouch.decomposition import FunkSVD
import logging

logging.basicConfig(level=logging.DEBUG)

In [3]:
movies = pd.read_csv("data/movies_clean.csv")
del movies['Unnamed: 0']
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [4]:
reviews = pd.read_csv("data/reviews_clean.csv")
del reviews['Unnamed: 0']
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
def create_train_test(X, order_by, training_size, testing_size, sample_size=10000):

    X = X.sort_values(by=order_by)
    sample = X.head(sample_size)
    
    X_train = sample.head(training_size)
    X_test = sample.tail(testing_size)
    
    return X_train, X_test

In [6]:
reviews_train, reviews_sub = create_train_test(reviews, 'date', 8000, 2000)

In [7]:
reviews_sub = reviews_train[["user_id", "movie_id", "rating"]]
print(reviews_sub.shape)
reviews_sub.head()

(8000, 3)


,user_id,movie_id,rating
498923,37287,2171847,6
442554,33140,444778,8
81920,6338,1411238,6
584570,43691,1496422,7
450669,33799,118799,5


In [8]:
movie_reviews = reviews_sub.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
movie_reviews.head()

movie_id,2844,8133,13427,14142,14538,14872,15163,15324,15400,17925,...,2625030,2635622,2638984,2645104,2645164,2646378,2649128,2735466,2748368,2750600
user_id,,,,,,,,,,,,,,,,,,,,,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
dec = FunkSVD(iters=20, learning_rate=0.005)
dec.fit(movie_reviews)

DEBUG:root:Optimization Statistics
DEBUG:root:Iterations | Mean Squared Error 
DEBUG:root:1 		 32.487816032900405
DEBUG:root:2 		 21.035193171715548
DEBUG:root:3 		 15.735274455033338
DEBUG:root:4 		 12.329859133124096
DEBUG:root:5 		 9.909243577547366
DEBUG:root:6 		 8.110429227731013
DEBUG:root:7 		 6.727067387345984
DEBUG:root:8 		 5.633422765343641
DEBUG:root:9 		 4.7502336849098965
DEBUG:root:10 		 4.025991696912126
DEBUG:root:11 		 3.4260525832286812
DEBUG:root:12 		 2.9260807675040468
DEBUG:root:13 		 2.508096699001118
DEBUG:root:14 		 2.158140431966431
DEBUG:root:15 		 1.8649366101674185
DEBUG:root:16 		 1.619152131724362
DEBUG:root:17 		 1.4129736794011967
DEBUG:root:18 		 1.2398388140637304
DEBUG:root:19 		 1.0942356294609863
DEBUG:root:20 		 0.9715376320412966


FunkSVD(iters=20, learning_rate=0.005)

In [10]:
dec.X_transformed.head()

movie_id,2844,8133,13427,14142,14538,14872,15163,15324,15400,17925,...,2625030,2635622,2638984,2645104,2645164,2646378,2649128,2735466,2748368,2750600
user_id,,,,,,,,,,,,,,,,,,,,,
8,5.306888,6.825450,2.809625,7.356904,7.720157,7.843165,7.378549,7.676670,6.663149,5.785640,...,7.833869,7.863101,7.747113,7.157693,2.614143,8.092246,9.002323,9.376915,9.213879,6.739189
46,4.477367,5.523129,2.827522,7.045461,5.981444,5.871112,6.698654,6.219678,5.106109,6.239526,...,6.539910,7.721697,6.235151,6.840723,2.901747,6.941529,9.648090,8.685711,8.898407,6.096754
48,4.758579,5.429144,2.457383,5.802946,6.253827,5.302381,6.346621,6.245085,5.599514,5.420772,...,5.835344,7.002587,6.783831,6.015311,2.476930,6.463247,8.067000,7.567510,7.524482,5.509522
51,3.831394,5.134340,2.079053,6.056749,5.941880,6.491951,5.855430,5.923523,4.917364,4.392844,...,6.501979,5.755670,5.424855,5.772155,1.941896,6.170819,6.789043,7.537643,7.407540,5.286644
66,5.522488,6.267969,3.082688,5.872883,6.482891,4.978496,6.337729,6.589894,6.226081,6.298219,...,5.404768,8.694628,8.376844,6.094181,3.039561,7.444189,9.892738,7.857919,7.975400,5.959095


In [11]:
dec.X_transformed.loc[8].sort_values(ascending=False).head(5).index.tolist()

[1255953, 1853728, 1707386, 111161, 1024648]

In [12]:
user_ids = [8, 46, 48, 51, 66]
dec.recommend(user_ids)

[[8, [1255953, 1853728, 1707386, 111161, 1024648]],
 [46, [1255953, 1045658, 1853728, 1623205, 454876]],
 [48, [1255953, 1045658, 1853728, 1772341, 454876]],
 [51, [1255953, 1853728, 1024648, 111161, 1790885]],
 [66, [1351685, 1675434, 1255953, 780504, 68646]]]